In [ ]:
import dash_helper as dash

results = dash.main('experiments/2024-02-02/results-for-2023-01-01-2023-12-31.csv',
               model1='experiments/models/train-until-2023-01-01_col_length=71.model',
               result2='experiments/2024-02-02_retrain90_predict2023/results-for-2023-01-01-2023-12-31.csv',
               model2='experiments/models/train-until-2023-12-27_col_length=71.model',
               orig_data='data/data_2024-02-02.csv', use_rolling=[True, True],
               use_feature_eng=[False, False])

preds1, columns1, model1, shap1 = results['results1']
preds2, columns2, model2, shap2 = results['results2']
orig_data = results['orig_data']
data1, data2 = orig_data[columns1], orig_data[columns2]

In [ ]:
target_col = 'F380 M2/M3'
TARGET_VALS = orig_data[[target_col]]

In [ ]:
dir(shap1)

In [ ]:
shap1.base_values

In [ ]:
import plotly.graph_objects as go

X_train = data1.loc['2023-01-01':]

# Extract the expected value (base value)
explainer, shap_values = dash.get_shap_values(X_train, model1)
expected_value = explainer.expected_value

# Create a bar plot for the SHAP values
feature_names = X_train.columns  # Replace with your feature names
shap_values_values = shap_values[0]
shap_plot = go.Bar(
    x=shap_values_values,
    y=feature_names,
    orientation='h',
    marker=dict(
        color=shap_values_values,
        colorscale='Viridis',  # You can choose other color scales
        showscale=True,
    ),
)

# Create a scatter plot for the expected value
expected_value_plot = go.Scatter(
    x=[expected_value],
    y=['Expected Value'],
    mode='markers',
    marker=dict(
        color='red',  # Change the color as desired
        size=15,
        symbol='x',
    ),
)

# Create the layout for the plot
layout = go.Layout(
    title='SHAP Force Plot',
    xaxis=dict(title='SHAP Value'),
    yaxis=dict(title='Feature'),
    showlegend=False,
)

# Create the Plotly figure
fig = go.Figure(data=[shap_plot, expected_value_plot], layout=layout)

# Show the figure
fig.show()


In [ ]:
import os
import re
import pandas as pd
import panel as pn
import param
import os
import holoviews as hv

import pandas as pd
import plotly.graph_objects as go

# perspective packages
import pickle
from pathlib import Path
import datetime as dt

pn.config.js_files["jquery"]="https://code.jquery.com/jquery-1.12.4.min.js"
pn.config.js_files["sparkline"]="https://cdnjs.cloudflare.com/ajax/libs/jquery-sparklines/2.1.2/jquery.sparkline.min.js"

pn.extension('tabulator', 'plotly', 'ipywidgets', 'perspective',  css_files=[pn.io.resources.CSS_URLS['font-awesome']], sizing_mode='stretch_width', omms='ipywidgets')

pn.config.js_files["sparkline_to_tabulator"]="https://cdn.jsdelivr.net/gh/MarcSkovMadsen/awesome-panel-assets@master/js/add_spark_lines_to_tabulator.min.js"


In [ ]:
import shap

X_train, X_test = data1.loc[:'2023-01-01'], data1.loc['2023-01-01':]


# Choose an instance to explain (e.g., the first instance)
instance_idx = 0
instance = pd.DataFrame(X_test.iloc[instance_idx]).T

# Calculate SHAP values
explainer = shap.Explainer(model1)
shap_values = explainer(instance)

In [ ]:
X_test

In [ ]:
from explainerdashboard import RegressionExplainer, ExplainerDashboard
import xgboost as xgb

model_name = 'experiments/models/model-partial-cols=70.model'

model = xgb.XGBRegressor(n_estimators=1500, max_depth=12, subsample=0.7, min_child_weight=9, learning_rate=0.041, tree_method='hist', xgb_model=model_name)
model.load_model(model_name)

# X_test.drop(columns='rolling_target5', inplace=True)
y_test = orig_data['F380 M2/M3']
# X_test.drop(columns='F380 M2/M3', inplace=True)

explainer = RegressionExplainer(model, data1, orig_data['F380 M2/M3'])

In [ ]:
explainer.shap

In [ ]:
from explainerdashboard import InlineExplainer

InlineExplainer(explainer).shap.overview()

In [ ]:
InlineExplainer(explainer).run()

In [ ]:
ExplainerDashboard(explainer, simple=True).run()

In [ ]:
import matplotlib.pyplot as plt

def create_shap_plot():
    explainer, shap_values = dash.get_shap_values(X_test, model1)

    # Assuming you want to visualize SHAP values for a specific feature (e.g., feature 0)
    feature_idx = 0
    shap.force_plot(explainer.expected_value, shap_values, X_test, matplotlib=False)  # Adjust X_test as needed

    # You can customize the SHAP plot further if needed

    plt.title(f'SHAP Summary Plot for Feature {feature_idx}')
    plt.xlabel('SHAP Value')
    plt.ylabel('Feature Value')
    
    # Use HoloViz's `pn.panel` to convert the Matplotlib plot to a panel-compatible object
    shap_plot = pn.panel(plt.gcf())

    return shap_plot


# Create a panel dashboard
dashboard = pn.Column(
    create_shap_plot(),
    pn.widgets.IntSlider(name='Select Instance Index', start=0, end=len(X_test) - 1, value=instance_idx)
)

# Define a callback to update the SHAP plot based on the selected instance index
def update_shap_plot(event):
    instance_idx = event.new
    shap_plot = create_shap_plot()

dashboard[1].param.watch(update_shap_plot, 'value')

# Show the dashboard
dashboard.servable()